In [71]:
import pandas as pd
import sys
import os
import json
sys.path.append('../')

In [184]:
df_parking = pd.read_csv('/Users/timlee/data/sf_parking/parkingrecords.csv')
ll_ids = pd.DataFrame(df_parking.groupby(['lat','lon']).size().reset_index())
ll_ids.columns = ['lat','lon','pid']
ll_ids['pid']= ll_ids.index
df_parking = df_parking.merge(ll_ids, how='left', on =['lat','lon'])

In [185]:
def gpsaddr_extract_json(one_addr):
    one_addr_details = {}
    one_addr_details['related_addr']  = []
    for item in one_addr:
        for subitem in item['address_components']:
            if subitem['types'] == ['street_number']:
                one_addr_details['street_no'] = subitem['long_name']
            if subitem['types'] == ['route']:
                one_addr_details['street_name'] = subitem['long_name']
            if 'neighborhood' in subitem['types']:
                one_addr_details['neighborhood'] = subitem['short_name']
            if 'postal_code' in subitem['types']:
                one_addr_details['zipcode'] = subitem['short_name']            
        if ('political' not in item['types']) and ('postal_code' not in item['types']):
            one_addr_details['related_addr'].append([item['formatted_address'],item['types']])
    if len(one_addr_details['related_addr']) > 1:
        one_addr_details['address_tags'] = [ x[1] for x in one_addr_details['related_addr'][1:]]
    return one_addr_details

In [186]:
search_dir = '../ref_data/gmaps_json/'
addr_collection = []
for fl in os.listdir(search_dir):
    if ('parking_json' in fl) and (fl != 'parking_json_0.json') :
        print(fl)
        with open(search_dir + fl,'r') as f:
            samp = f.read()
            samp = samp.replace('}{"lat','}\n{"lat')
            lines = samp.split('\n')
            jsons = [json.loads(ln) for ln in lines]            
            
        for jsn in jsons:
            lat = jsn['lat']
            lng = jsn['lng']
            one_addr_details = gpsaddr_extract_json(jsn['data'])
            one_addr_details['orig_lat'] = float(lat)
            one_addr_details['orig_lng'] = float(lng)
            one_addr_details['Search_GPS'] = lat+','+lng
            addr_collection.append(one_addr_details)
        print(len(addr_collection))

parking_json_21200.json
100
parking_json_27100.json
200
parking_json_20000.json
300
parking_json_26300.json
400
parking_json_18100.json
500
parking_json_23600.json
600
parking_json_9300.json
700
parking_json_25500.json
800
parking_json_22800.json
900
parking_json_25900.json
1000
parking_json_22400.json
1100
parking_json_19300.json
1200
parking_json_8100.json
1300
parking_json_24700.json
1400
parking_json_5600.json
1500
parking_json_12700.json
1600
parking_json_14400.json
1700
parking_json_3500.json
1800
parking_json_13900.json
1900
parking_json_4800.json
2000
parking_json_3900.json
2100
parking_json_14800.json
2200
parking_json_4400.json
2300
parking_json_13500.json
2400
parking_json_15600.json
2500
parking_json_2700.json
2600
parking_json_10300.json
2700
parking_json_7200.json
2800
parking_json_1100.json
2900
parking_json_16000.json
3000
parking_json_500.json
3100
parking_json_900.json
3200
parking_json_11100.json
3300
parking_json_6000.json
3400
parking_json_17200.json
3500
parking_j

28120


In [156]:
df = pd.DataFrame(addr_collection)
df.loc[df['address_tags'].isna()==False,'address_tags'] = df.loc[df['address_tags'].isna()==False,'address_tags'].map(lambda x : ','.join(x[0]))
df['related_addr'] = df['related_addr'].astype(str)
df.to_feather('../ref_data/gmaps_df_parking.feather')

In [157]:
parking_gps = df.copy()

In [158]:
parking_gps.shape

(28211, 9)

In [159]:
parking_gps.head()

,Search_GPS,address_tags,neighborhood,orig_lat,orig_lng,related_addr,street_name,street_no,zipcode
0,"37.789004,-122.417333",NaN,Lower Nob Hill,37.789004,-122.417333,"[['897 Hyde St, San Francisco, CA 94109, USA',...",Hyde Street,897,94109
1,"37.789005,-122.422216","establishment,general_contractor,local_governm...",Western Addition,37.789005,-122.422216,"[['1455 Van Ness Ave, San Francisco, CA 94109,...",Van Ness Avenue,1400,94109
2,"37.789005,-122.409838",street_address,Lower Nob Hill,37.789005,-122.409838,"[['575 Sutter St, San Francisco, CA 94102, USA...",Mason Street,530,94108
3,"37.789005,-122.402346","bus_station,establishment,point_of_interest,tr...",Financial District,37.789005,-122.402346,"[['19-31 Post St, San Francisco, CA 94104, USA...",Post Street,19-31,94104
4,"37.789007,-122.434913","bus_station,establishment,point_of_interest,tr...",Pacific Heights,37.789007,-122.434913,"[['2450 California St, San Francisco, CA 94115...",California Street,2450,94115


In [139]:
parking_gps[parking_gps['orig_lng'] == -122.414177]

,Search_GPS,address_tags,neighborhood,orig_lat,orig_lng,related_addr,street_name,street_no,zipcode


In [140]:
w2 = str(37.7716371)
w = str(37.7716370)
len(w2)

10

In [188]:
df_parking['str_lat'] = df_parking['lat'].round(6).astype(str)
df_parking['str_lat'] = df_parking['str_lat'].map(lambda x : x+'0'*(9-len(x)) if len(x)<9 else x)
df_parking['str_lng'] = df_parking['lon'].round(6).astype(str)
df_parking['str_lng'] = df_parking['str_lng'].map(lambda x : x+'0'*(11-len(x)) if len(x)<11 else x)
df_parking['joinkey'] = df_parking['str_lat'].str[:9] + ',' + df_parking['str_lng'].str[:11] 
df_parking['joinkey'] = df_parking['joinkey'].str.strip()
df_parking.drop(columns=['str_lat','str_lng'], inplace=True)
df_parking.head()

,lat,lon,datetime,pid,joinkey
0,37.771637,-122.437112,1471967637400,8710,"37.771637,-122.437112"
1,37.777229,-122.465370,1459821377952,11364,"37.777229,-122.465370"
2,37.771172,-122.437683,1461349554900,8478,"37.771172,-122.437683"
3,37.777328,-122.465012,1462252229600,11458,"37.777328,-122.465012"
4,37.778664,-122.464111,1463364111300,12038,"37.778664,-122.464111"


In [207]:
stage1mrg = df_parking.merge(parking_gps, how='left', left_on='joinkey', right_on=['Search_GPS']).sort_values('Search_GPS')

In [210]:
stage1mrg['Search_GPS'].isna().sum()

0

### Follow up

In [153]:
follow_up = stage1mrg[stage1mrg['Search_GPS'].isna()].groupby(['lat','lon','pid']).size().reset_index()

In [152]:
import googlemaps
with open('/Users/timlee/Dropbox/keys/google_api_key.txt','r') as f:
    gmap_api_key = f.read()

gmaps = googlemaps.Client( key = gmap_api_key)
output = []
raw_json = []

for lat, lng, pid, ct in follow_up.values:
    # print('Reverse pulling ... %s %s' %(lat,lng))
    geocode_result = gmaps.reverse_geocode((lat,lng))
    store_json = {
        'lat' : lat,
        'lng' : lng,
        'pid' : pid,
        'data': geocode_result
    }
    raw_json.append(store_json)

In [172]:
raw_json[0]

{'data': [{'address_components': [{'long_name': '968a',
     'short_name': '968a',
     'types': ['street_number']},
    {'long_name': 'Geneva Avenue',
     'short_name': 'Geneva Ave',
     'types': ['route']},
    {'long_name': 'Crocker-Amazon',
     'short_name': 'Crocker-Amazon',
     'types': ['neighborhood', 'political']},
    {'long_name': 'San Francisco',
     'short_name': 'SF',
     'types': ['locality', 'political']},
    {'long_name': 'San Francisco County',
     'short_name': 'San Francisco County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'California',
     'short_name': 'CA',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '94112', 'short_name': '94112', 'types': ['postal_code']},
    {'long_name': '3403',
     'short_name': '3403',
     'types': ['postal_code_suffix']}],
   'formatted_address': '968a Gen

In [191]:
follow_up = []
for i, rj in enumerate(raw_json):
    lat = rj['lat']
    lng = rj['lng']
    pid = rj['pid']
    one_addr_details = gpsaddr_extract_json(rj['data'])
    one_addr_details['orig_lat'] = float(lat)
    one_addr_details['orig_lng'] = float(lng)
    one_addr_details['pid'] = int(pid)
    one_addr_details['Search_GPS'] = str(lat)+','+str(lng)
    addr_collection.append(one_addr_details)
    follow_up.append(one_addr_details)

In [195]:
follow_up_df = pd.DataFrame(follow_up)
follow_up_df.shape

(91, 10)

In [202]:
follow_up_df.head()

,Search_GPS,address_tags,neighborhood,orig_lat,orig_lng,pid,related_addr,street_name,street_no,zipcode
0,"37.7158145,-122.4400187","[[bus_station, establishment, point_of_interes...",Crocker-Amazon,37.715815,-122.440019,147,"[[968a Geneva Ave, San Francisco, CA 94112, US...",Geneva Avenue,968a,94112
1,"37.7392535,-122.4699492",NaN,West of Twin Peaks,37.739253,-122.469949,2385,"[[2600-2616 14th Ave, San Francisco, CA 94127,...",14th Avenue,2600-2616,94127
2,"37.7527195,-122.4113014",NaN,Mission District,37.752719,-122.411301,4166,"[[2950 24th St, San Francisco, CA 94110, USA, ...",24th Street,2950,94110
3,"37.7534865,-122.420683399",NaN,Mission District,37.753487,-122.420683,4318,"[[1201-1215 Valencia St, San Francisco, CA 941...",Valencia Street,1201-1215,94110
4,"37.7542535,-122.477939201",NaN,Sunset District,37.754253,-122.477939,4584,"[[1322 Noriega St, San Francisco, CA 94122, US...",Noriega Street,1322,94122


In [193]:
backfill_dict = {pid: srch for pid, srch in follow_up_df[['pid','Search_GPS']].values}

In [206]:
stage1mrg.drop(columns=['related_addr','address_tags'], inplace=True)

In [209]:
for row in follow_up_df.values:
    pid = row[-5]
    mask = stage1mrg['pid']==pid
    stage1mrg.loc[mask, 'Search_GPS'] = row[0]
    stage1mrg.loc[mask, 'neighborhood'] = row[2]
    stage1mrg.loc[mask, 'orig_lat'] = row[3]
    stage1mrg.loc[mask, 'orig_lng'] = row[4]
    stage1mrg.loc[mask, 'street_name'] = row[7]
    stage1mrg.loc[mask, 'street_no'] = row[8]
    stage1mrg.loc[mask, 'zipcode'] = row[9]
    

In [212]:
stage1mrg.drop(columns=['address_tags', 'related_addr'], inplace=True)

In [216]:
stage1mrg.reset_index(inplace=True)

In [217]:
stage1mrg.to_feather('../ref_data/gmaps_df_parking.feather')